In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    if np.sum([i**2 for i in vec]) - (vec[0])**2 == 0:
        H = np.eye(vec.size)
    else:
        y = np.zeros_like(vec)
        y[0] = np.linalg.norm(vec)
        u = (vec - y) / np.linalg.norm(vec - y)
        H = np.eye(u.size) - 2 * np.outer(u, u)
    return y, H    

In [3]:
# Test (marked)

v = np.array([1, 5, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [4]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    q = np.eye(m)
    
    for i in range(n):
        x = a1[i:, i]
        q_i = np.zeros((m, m))
        q_i[i:, i:] += householder(x)[1]
        for j in range (i):
            q_i[j][j] = 1
        a1 = q_i @ a1
        q = q @ q_i.T
    r = a1
    return (q, r)  

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [7]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [9]:
print('a', a, sep = '\n')
print('q*r', q @ r, sep = '\n')
print('q', qr_decomp(a)[0], 'qq', qq, sep = '\n')
print('r', qr_decomp(a)[1], 'rr', rr, sep = '\n')

a
[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]
q*r
[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]
q
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
qq
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.752227

As you can see, the answers can be different only in signs, which can not affect the decomposed matrix

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [10]:
def qr2(a):
    a1 = np.array(a, copy=True, dtype=float)
    a_res = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    u_res = np.zeros((m,n))
    for i in range (n):
        x = a1[:,0]
        y = np.zeros(len(x))
        y[0] = np.linalg.norm(x)
        u = (x - y) / np.linalg.norm(x - y)
        u0 = np.zeros(m)
        u0[i:] = u
        u_res[:,i] = u0
        u1 = u0[:,None].T @ a_res
        a_res = a_res - 2 * (u0[:,None] @ u1)
        a1 = a_res[i+1:,i+1:]
    
    return a_res, u_res

In [11]:
def QT(a,u):
    """Uses the reflection vertices to multiply a matrix with  𝐐𝑇.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    u : ndarray, shape(m, n)
        Reflection vectors
    
    Returns
    -------
    a_res : Multiplied matrix
    
    """
   
    a_res = np.array(a, copy=True, dtype=float)
    m, n = a.shape
    for i in range (0,n):
        d = (u[:,i])[:,None].T @ a_res
        a_res = a_res - 2 * ((u[:,i])[:,None] @ d)
    return a_res

Q - ортоганальная, поэтому 𝐐𝑇, умноженная на A, тоже должна давать R

In [12]:
a = rndm.rand(10, 7)
r, u = qr2(a)
print('r', r,'QT*A', QT(a, u), sep = '\n')

r
[[ 1.62313602  1.67955203  1.15198958  1.21019174  1.39830052  1.10759776
   1.23423964]
 [ 0.          1.06596437  0.77917752  0.73458533  0.41032078  0.93446484
   0.45133467]
 [ 0.         -0.          0.98017165  0.400185    0.24354926  0.57972376
   0.43755821]
 [-0.          0.         -0.          1.34254044  0.08663159  0.0164154
   0.48645789]
 [-0.         -0.         -0.         -0.          1.23531039  0.5539262
   0.67044186]
 [-0.         -0.         -0.         -0.          0.          0.53917195
   0.54014939]
 [ 0.          0.          0.         -0.         -0.         -0.
   0.57319371]
 [ 0.         -0.         -0.          0.          0.         -0.
   0.        ]
 [ 0.          0.         -0.         -0.          0.          0.
   0.        ]
 [ 0.         -0.         -0.          0.         -0.         -0.
   0.        ]]
QT*A
[[ 1.62313602  1.67955203  1.15198958  1.21019174  1.39830052  1.10759776
   1.23423964]
 [ 0.          1.06596437  0.77917752  0.734585